# Proyecto del Día 14 - Gestión Segura de Datos de Clientes

Este notebook implementa un sistema básico de gestión y análisis de datos de clientes, aplicando técnicas de pseudonimización, anonimización y balanceo de datos.


## Objetivo
El proyecto consiste en desarrollar un sistema de gestión y análisis de datos de clientes que implemente técnicas de pseudonimización, anonimización y balanceo de datos para proteger la privacidad de los usuarios y asegurar un análisis equitativo.


## Consignas del Proyecto
1. Cargar los datos de clientes provistos en un archivo CSV.
2. Aplicar técnicas de anonimización al campo `direccion`, `edad` y `salario` para proteger la identidad de los clientes.
3. Aplicar técnicas de pseudonimización al campo `nombre`.
4. Implementar una función de balanceo de datos para asegurar la representación equitativa de diversas categorías en el análisis posterior.
5. Realizar un análisis simple de los datos para identificar tendencias o patrones que podrían ser útiles para decisiones de negocios o marketing.


Este proyecto te permitirá aplicar prácticas esenciales de privacidad de datos y técnicas de manipulación de datos en un entorno práctico, preparándote para desafíos similares en entornos profesionales.

¡Mucha suerte y que te diviertas!

In [9]:
import os
import uuid

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dotenv import load_dotenv

In [4]:
load_dotenv('.env')

True

### 1. Cargar los datos de clientes provistos en un archivo CSV.

In [5]:
df: pd.DataFrame = pd.read_csv(os.environ['PROJECT_DATA'])

### 2. Aplicar técnicas de anonimización al campo `direccion`, `edad` y `salario` para proteger la identidad de los clientes.

In [ ]:
# Anonimizar el campo dirección por simple supresión
# (asumiendo que no es relevante para el estudio)
df.drop(['direccion'], axis=1, inplace=True)

In [ ]:
# Anonimizar el campo edad por redondeo o truncado
# adaptando las edades a sus respectivas décadas
df['edad'] = (df['edad'] // 10) * 10

In [8]:
# Anonimizar el campo salario con ruido aleatorio
noise: np.ndarray = np.random.normal(0, 10000, size=df['salario'].shape)
df['salario'] += noise

### 3. Aplicar técnicas de pseudonimización al campo `nombre`.

In [10]:
# Pseudonomización de los valores del campo nombre
# usando un identificador único universal (UUID) para cada valor
uuid_container: list[str] = []

for _ in range(len(df)):
    uuid_container.append(str(uuid.uuid4()))

df['uuid'] = uuid_container
df.drop(['nombre'], axis=1, inplace=True)

A partir de aquí, ya parece seguro mostrar el contenido de `df` para los siguientes pasos. 😁

In [11]:
df.head()

,edad,categoria,salario,uuid
0,60,0,74189.466539,1908635c-7c0a-4db0-82cc-bcd0971c4449
1,60,1,50776.658566,9d3178c7-c3e3-4def-9803-b8c799f0aa7b
2,10,0,66739.991400,14b51fca-4094-4dd8-a9f4-4e74b9d00184
3,20,1,23795.264394,e43c1f82-f14b-45ab-861e-705b0e8a2f0c
4,20,1,32557.826303,aaa3cb4c-f1d7-4593-831f-ad6c86982fb9
